<a href="https://colab.research.google.com/github/sheikmohdimran/hackathon/blob/master/NLP_CRF_CADEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#In Python2
#!unzip -q CADEC.zip
#!rm CADEC.zip; rm CADEC.v1.zip; rm -rf CSIRO_DAP_Metadata
#!unzip CADEC.v2.zip
#!mv cadec/original/* cadec/text/
#!cp  cadec/text/DICLOFENAC-SODIUM.7.ann  cadec/text/DICLOFENAC-SODIUM.7.ann.orig
#!sed -i 's/421/411/g'  cadec/text/DICLOFENAC-SODIUM.7.ann  #Change annotation lines
#!rm -rf VOLTAREN-XR.9*; rm -rf LIPITOR.40* #No entries with these files
#!git clone https://github.com/spyysalo/standoff2conll
#!python standoff2conll/standoff2conll.py -i -s IO cadec/text  > cadec/text.tsv

In [0]:
import pandas as pd
import numpy as np

with open('cadec_text.tsv') as f:
    content = f.readlines()

indices = [i for i, x in enumerate(content) if x.startswith("# doc_id")]
df1 = pd.DataFrame([sub.split("\t") for sub in content], columns=('words','tags'))
indices1 = [i for i, x in enumerate(content) if x.startswith("\n")]
a=0
indices1.insert(0, a)
k=0
j = None
for i in indices1:
    if j != None:
      df1.loc[j:i, ("doc_id")] = k
    j = i
    k += 1

df1 = df1.replace('\n','', regex=True)
df1 = df1.replace('I-Symptom','O')
df1 = df1.replace('I-Disease','O')
df1 = df1.replace('I-Finding','O')
df1 = df1.drop(df1.index[list(indices)])
df1.dropna(inplace=True)
df1 = df1.reset_index(drop=True)

In [0]:
from tqdm import tqdm, trange

class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["words"].values.tolist(),
                                                           s["tags"].values.tolist())]
        self.grouped = self.data.groupby("doc_id").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None


In [0]:
getter = SentenceGetter(df1)
import nltk
nltk.download('averaged_perceptron_tagger')
data = []
for i, doc in enumerate(getter.sentences):

    # Obtain the list of tokens in the document
    tokens = [t for t, tags in doc]

    # Perform POS tagging
    tagged = nltk.pos_tag(tokens)

    # Take the word, POS tag, and its label
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features


In [0]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [0]:
!pip install python-crfsuite


     |████████████████████████████████| 757kB 2.7MB/s 


In [0]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 25937
Seconds required: 0.198

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 58209.507295
Feature norm: 1.000000
Error norm: 50499.890540
Active features: 25492
Line search trials: 1
Line search step: 0.000005
Seconds required for this iteration: 0.193

***** Iteration #2 *****
Loss: 50092.670998
Feature norm: 0.853034
Error norm: 36715.729118
Active features: 24304
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.100

***** Iteration #3 *****
Loss: 46341.796177
Feature norm: 0.697910
Error norm: 47848.383390
Active features: 23996
Line search trials: 1
Line search step: 1.000000
Seconds required for

In [0]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

after (O)
reading (O)
this (O)
, (O)
i (O)
am (O)
sure (O)
that (O)
lipitor (I-Drug)
is (O)
causing (O)
all (O)
this (O)
. (O)


In [0]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
tags_vals = list(set(df1["tags"].values))
labels = {t: i for i, t in enumerate(tags_vals)}
# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["I-ADR", "I-Drug","O"]))


              precision    recall  f1-score   support

       I-ADR       0.94      0.96      0.95     21399
      I-Drug       0.97      0.85      0.91       422
           O       0.76      0.67      0.72      3715

   micro avg       0.92      0.92      0.92     25536
   macro avg       0.89      0.83      0.86     25536
weighted avg       0.92      0.92      0.92     25536



In [0]:
from sklearn.metrics import f1_score
print(f1_score(
    truths, predictions, average='micro'))	


0.9194470551378445
